In [1]:
from statsmodels.tsa.arima_model import ARIMA
import matplotlib.pyplot as plt
import pandas as pd
from pandas import read_csv
from pandas import datetime
from matplotlib import pyplot
def parser(x):
    return datetime.strptime(x , '%d/%m/%Y')
ColumnList=[]

series= read_csv('DP26-skus monthly sales (FILTERED2).csv', parse_dates=['Date'] , index_col='Date', date_parser=parser)
for columns in series:
    ColumnList.append(columns)
print (ColumnList)
print(series.head())
series.plot()
pyplot.show()

c:\users\admin\appdata\local\programs\python\python36-32\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


['1000403940202', "'1000403940203", "'1000403940206", "'1000600164107", "'1000600164110", "'1001600066003", "'1001600066005", "'10020000611502", "'1002000063302", "'1002000063305", "'1002000063306", "'1002000063307", "'1002003933701", "'1002003933702", "'1002003933704", "'1002003933705", "'10020044127401", "'10020044127402", "'10020044127403", "'1004200045501", "'1004200045503", "'1004400337902", "'1004400337905", "'1004400337906", "'1005200524703", "'1005200524705", "'1005200524708", "'1005204362301", "'1005204362302", "'1005204362303", "'1005204362304", "'1011604219701", "'1011604219703", "'1011604219704", "'1011604219706", "'1011604229801", "'1011604229802", "'1011604239901", "'2000100012001", "'2000100012006", "'2000100012009", "'2000100012010", "'2000100142206", "'2000100142210", "'2000100321306", "'2001100126401", "'2001100126405", "'2001100126406", "'2001100126407", "'2001100126408", "'2001100126409", "'2001100126410", "'2001100136901", "'2001100136903", "'2001100136904", "'2001

<Figure size 640x480 with 1 Axes>

In [3]:
model={}
ColName=[]
simpleExpo_mape=[]
simpleExpo_conf=[]
simpleExpo_sum_error=[]
simpleExpo_forecast_sum=[]
model['simpleExpo_mape']=simpleExpo_mape
model['simpleExpo_conf']=simpleExpo_conf
model['simpleExpo_absolute_error']=simpleExpo_sum_error
model['simpleExpo forecast sum']=simpleExpo_forecast_sum
model['ColName']=ColName

In [4]:
model2={}
simpleExpo_forecast=[]
SKU=[]
model2['simpleExpo_forecast']=simpleExpo_forecast
model2['SKU']=SKU

In [5]:
import numpy as np
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return (((np.sum(np.abs(y_true - y_pred))) / (np.sum(y_pred))) * 100)

In [ ]:
def movingavg(series,l,u):
    min = 999999999999999
    for w in range (l,u+1):
        result=[]
        for j in range(w):
            result.append(series[j])
        for i in range (w,len(series)):
            result.append((sum(series[i-w:i]))/w)
        mape=mean_absolute_percentage_error(series[24:],result[24:])
        #print result
        if (0<=mape<min):
            min=mape
            minW=w
            minr=result
        print ('Window-size {} - mape {}'.format(w,mape))
    absolute_error=(min/100)*sum(minr[24:])
    forecast_sum=sum(minr[24:])
    print ('Min mape is {} - window size {} - absolute error {} - forecast sum {}'.format(mape,minW,absolute_error,forecast_sum))
    movingavg_mape.append(min)
    movingavg_conf.append(minW)
    movingavg_sum_error.append(absolute_error)
    movingavg_forecast_sum.append(forecast_sum)
    movingavg_forecast.extend(minr)
    return minr
def exponential_smoothing(series, alphaL,alphaU):
    alphal=int(10*alphaL)
    alphau=int(10*alphaU)
    min = 999999999999999
    
    for alpha in range (alphal, alphau+1):
        result=[]
        result = [series[0]] # first value is same as series
        for n in range(1, (len(series))):
            result.append((alpha/10.0) * series[n-1] + (1 - (alpha/10.0)) * result[n-1])
        mape=mean_absolute_percentage_error(series[24:],result[24:])
        #print result
        if (0<=mape<min):
            min=mape
            minalpha=(alpha/10.0)
            minr=result
        print ('alpha {} - mape {}'.format((alpha/10.0),mape))
    absolute_error=(min/100)*sum(minr[24:])
    forecast_sum=sum(minr[24:])
    print ('best alpha {} - mape is {} - absolute error {} - forecast sum {}'.format(minalpha,min,absolute_error,forecast_sum))
    simpleExpo_mape.append(min)
    simpleExpo_conf.append(minalpha)
    simpleExpo_sum_error.append(absolute_error)
    simpleExpo_forecast_sum.append(forecast_sum)
    simpleExpo_forecast.extend(minr)
    return minr

def double_exponential_smoothing(series, alphaL, alphaU, betaL, betaU):
    alphal=int(10*alphaL)
    alphau=int(10*alphaU)
    betal=int(10*betaL)
    betau=int(10*betaU)
    min = 999999999999999
    
    for alpha in range (alphal, alphau+1):
        result=[]
        for beta in range (betal, betau+1):
            result = [series[0], series[0]]
            for n in range(2, len(series)):
                if n == 2:
                    level, trend = series[0], series[1] - series[0]
                if n <= len(series): # we are forecasting
                    value = series[n-1]
                last_level, level = level, (alpha/10.0)*value + (1-(alpha/10.0))*(level+trend)
                trend = (beta/10.0)*(level-last_level) + (1-(beta/10.0))*trend
                result.append(level+trend)
            mape=mean_absolute_percentage_error(series[24:],result[24:])
            #print result
            if (0<=mape<min):
                min=mape
                minalpha=(alpha/10.0)
                minbeta=(beta/10.0)
                minr=result
            print ('alpha {} - beta {} - mape {}'.format((alpha/10.0),(beta/10.0),mape))
    absolute_error=(min/100)*sum(minr[24:])
    forecast_sum=sum(minr[24:])
    print ('min alpha {} - min beta {} - mape {} - absolute error {} - forecast sum {}'.format(minalpha,minbeta,min,absolute_error,forecast_sum))
    DoubleExpo_mape.append(min)
    DoubleExpo_conf.append([minalpha,minbeta])
    DoubleExpo_sum_error.append(absolute_error)
    DoubleExpo_forecast_sum.append(forecast_sum)
    DoubleExpo_forecast.extend(minr)
    return minr

def M_initial_trend(series, slen):
    sum = 0.0
    for i in range(slen):
        sum += float(series[i+slen] - series[i]) / slen
    return sum / slen
def M_initial_seasonal_components(series, slen):
    seasonals = {}
    season_averages = []
    n_seasons = int(len(series)/slen)
    # compute season averages
    for j in range(n_seasons):
        season_averages.append(sum(series[slen*j:slen*j+slen])/float(slen))
    # compute initial values
    for i in range(slen):
        sum_of_vals_over_avg = 0.0
        for j in range(n_seasons):
            sum_of_vals_over_avg += series[slen*j+i]-season_averages[j]
        seasonals[i] = sum_of_vals_over_avg/n_seasons
    return seasonals
def M_triple_exponential_smoothing(series, slenL, slenU, alphaL, alphaU, betaL, betaU, gammaL, gammaU, n_preds):
    alphal=int(10*alphaL)
    alphau=int(10*alphaU)
    betal=int(10*betaL)
    betau=int(10*betaU)
    gammal=int(10*gammaL)
    gammau=int(10*gammaU)
    min = 999999999999999
    min2=999999999999999
    for slen in range (slenL, slenU+1):
        try:
            for alpha in range (alphal, alphau+1):
                for beta in range (betal, betau+1):
                    for gamma in range (gammal, gammau+1):
                        result=[]
                        seasonals = M_initial_seasonal_components(series, slen)
                        for i in range(len(series)+n_preds):
                            if i == 0: # initial values
                                smooth = series[0]
                                trend = M_initial_trend(series, slen)
                                result.append(series[0])
                                continue
                            if i >= len(series): # we are forecasting
                                m = i - len(series) + 1
                                result.append((smooth + m*trend) * seasonals[i%slen])
                            else:
                                val = series[i-1]
                                last_smooth, smooth = smooth, (alpha/10.0)*(val/seasonals[i%slen]) + (1-(alpha/10.0))*(smooth+trend)
                                trend = (beta/10.0) * (smooth-last_smooth) + (1-(beta/10.0))*trend
                                seasonals[i%slen] = (gamma/10.0)*(val/smooth) + (1-(gamma/10.0))*seasonals[i%slen]
                                result.append((smooth+trend)*seasonals[i%slen])
                        mape=mean_absolute_percentage_error(series[24:],result[24:])
                        #print result
                        if (0<=mape<min):
                            min=mape
                            minalpha=(alpha/10.0)
                            minbeta=(beta/10.0)
                            mingamma=(gamma/10.0)
                            minr=result
                        print ('alpha {} - beta {} - gamma {} - mape {}'.format((alpha/10.0),(beta/10.0),(gamma/10.0),mape))
            print('best alpha {} - best beta {} - best gamma {} - mape {} - slen {}'.format(minalpha,minbeta,mingamma,min,slen))
        except:
            continue
        if (0<=min<min2):
            min2=min
            bestalpha=minalpha
            bestbeta=minbeta
            bestgamma=mingamma
            bestslen=slen
            bestr=minr
    absolute_error=(min2/100)*sum(bestr[24:])
    forecast_sum=sum(bestr[24:])
    print ('A {} - B {} - G {} - M {} - S {} - AE {} - FS {}'.format(bestalpha,bestbeta,bestgamma,min2,bestslen,absolute_error,forecast_sum))
    M_tripleExpo_mape.append(min2)
    M_tripleExpo_conf.append([bestalpha,bestbeta,bestgamma])
    M_tripleExpo_sum_error.append(absolute_error)
    M_tripleExpo_forecast_sum.append(forecast_sum)
    M_tripleExpo_forecast.extend(bestr)
    return bestr

def A_initial_trend(series, slen):
    sum = 0.0
    for i in range(slen):
        sum += float(series[i+slen] - series[i]) / slen
    return sum / slen
def A_initial_seasonal_components(series, slen):
    seasonals = {}
    season_averages = []
    n_seasons = int(len(series)/slen)
    # compute season averages
    for j in range(n_seasons):
        season_averages.append(sum(series[slen*j:slen*j+slen])/float(slen))
    # compute initial values
    for i in range(slen):
        sum_of_vals_over_avg = 0.0
        for j in range(n_seasons):
            sum_of_vals_over_avg += series[slen*j+i]-season_averages[j]
        seasonals[i] = sum_of_vals_over_avg/n_seasons
    return seasonals
def A_triple_exponential_smoothing(series, slenL, slenU, alphaL, alphaU, betaL, betaU, gammaL, gammaU, n_preds):
    alphal=int(10*alphaL)
    alphau=int(10*alphaU)
    betal=int(10*betaL)
    betau=int(10*betaU)
    gammal=int(10*gammaL)
    gammau=int(10*gammaU)
    min = 999999999999999
    min2=999999999999999
    for slen in range (slenL, slenU+1):
        try:
            for alpha in range (alphal, alphau+1):
                for beta in range (betal, betau+1):
                    for gamma in range (gammal, gammau+1):
                        result=[]
                        seasonals = A_initial_seasonal_components(series, slen)
                        for i in range(len(series)+n_preds):
                            if i == 0: # initial values
                                smooth = series[0]
                                trend = A_initial_trend(series, slen)
                                result.append(series[0])
                                continue
                            if i >= len(series): # we are forecasting
                                m = i - len(series) + 1
                                result.append((smooth + m*trend) + seasonals[i%slen])
                            else:
                                val = series[i-1]
                                last_smooth, smooth = smooth, (alpha/10.0)*(val-seasonals[i%slen]) + (1-(alpha/10.0))*(smooth+trend)
                                trend = (beta/10.0) * (smooth-last_smooth) + (1-(beta/10.0))*trend
                                seasonals[i%slen] = (gamma/10.0)*(val-smooth) + (1-(gamma/10.0))*seasonals[i%slen]
                                result.append(smooth+trend+seasonals[i%slen])
                        mape=mean_absolute_percentage_error(series[24:],result[24:])
                        #print result
                        if (0<=mape<min):
                            min=mape
                            minalpha=(alpha/10.0)
                            minbeta=(beta/10.0)
                            mingamma=(gamma/10.0)
                            minr=result
                        print ('alpha {} - beta {} - gamma {} - mape {}'.format((alpha/10.0),(beta/10.0),(gamma/10.0),mape))
            print('best alpha {} - best beta {} - best gamma {} - mape {} - slen {}'.format(minalpha,minbeta,mingamma,min,slen))
        except:
            continue
        if (0<=min<min2):
            min2=min
            bestalpha=minalpha
            bestbeta=minbeta
            bestgamma=mingamma
            bestslen=slen
            bestr=minr
    absolute_error=(min2/100)*sum(bestr[24:])
    forecast_sum=sum(bestr[24:])
    print ('A {} - B {} - G {} - M {} - S {} - AE {} - FS {}'.format(bestalpha,bestbeta,bestgamma,min2,bestslen,absolute_error,forecast_sum))
    A_tripleExpo_mape.append(min2)
    A_tripleExpo_conf.append([bestalpha,bestbeta,bestgamma])
    A_tripleExpo_sum_error.append(absolute_error)
    A_tripleExpo_forecast_sum.append(forecast_sum)
    A_tripleExpo_forecast.extend(bestr)
    return bestr

In [9]:
def forecasting ():
    for i in range(len(ColumnList)):
        ColName.append(ColumnList[i])
        for ik in range(0,36):
            SKU.append(ColumnList[i])
        print (ColumnList[i])
        print (i)
        print ("")
        exponential_smoothing(series[ColumnList[i]],0.1,0.9)

In [10]:
forecasting()

1000403940202
0

alpha 0.1 - mape 40.1243762548739
alpha 0.2 - mape 37.41990296993081
alpha 0.3 - mape 39.431192521019035
alpha 0.4 - mape 40.709255242832384
alpha 0.5 - mape 41.856411658968845
alpha 0.6 - mape 43.007235127657765
alpha 0.7 - mape 44.3234154184798
alpha 0.8 - mape 45.80541395808469
alpha 0.9 - mape 47.36692206620535
best alpha 0.2 - mape is 37.41990296993081 - absolute error 13126.050430736901 - forecast sum 35077.72438983738
'1000403940203
1

alpha 0.1 - mape 19.79111593872228
alpha 0.2 - mape 20.52567151167921
alpha 0.3 - mape 20.970567144930392
alpha 0.4 - mape 21.16271447553269
alpha 0.5 - mape 21.69946771488846
alpha 0.6 - mape 23.36784110439143
alpha 0.7 - mape 25.4013755539124
alpha 0.8 - mape 27.235417848918047
alpha 0.9 - mape 28.76803914113932
best alpha 0.1 - mape is 19.79111593872228 - absolute error 15742.146627315395 - forecast sum 79541.48051103637
'1000403940206
2

alpha 0.1 - mape 51.669079012823694
alpha 0.2 - mape 38.62607652739341
alpha 0.3 - mape 35

alpha 0.3 - mape 134.87034159743766
alpha 0.4 - mape 134.1103375980131
alpha 0.5 - mape 130.8370598132857
alpha 0.6 - mape 126.64103090649942
alpha 0.7 - mape 122.24408546570697
alpha 0.8 - mape 117.87302812114345
alpha 0.9 - mape 113.5762217391378
best alpha 0.9 - mape is 113.5762217391378 - absolute error 102477.93671142514 - forecast sum 90228.33753600006
'1004400337906
23

alpha 0.1 - mape 159.52868452349927
alpha 0.2 - mape 151.07464421327504
alpha 0.3 - mape 142.30782507191395
alpha 0.4 - mape 130.81891251907126
alpha 0.5 - mape 117.79038600769918
alpha 0.6 - mape 112.95547057691957
alpha 0.7 - mape 109.52187334056333
alpha 0.8 - mape 106.1629716223371
alpha 0.9 - mape 102.89351718043514
best alpha 0.9 - mape is 102.89351718043514 - absolute error 41477.10631516457 - forecast sum 40310.709023999916
'1005200524703
24

alpha 0.1 - mape 86.91179472653765
alpha 0.2 - mape 97.75871292243751
alpha 0.3 - mape 101.52603943563933
alpha 0.4 - mape 101.50725833128232
alpha 0.5 - mape 100.07

alpha 0.5 - mape 120.40732825329098
alpha 0.6 - mape 108.282683755692
alpha 0.7 - mape 102.28278143674554
alpha 0.8 - mape 98.54788868157483
alpha 0.9 - mape 94.8286697918436
best alpha 0.9 - mape is 94.8286697918436 - absolute error 22576.993208920372 - forecast sum 23808.193512024005
'2001100126401
45

alpha 0.1 - mape 47.18228759349405
alpha 0.2 - mape 48.85134037511429
alpha 0.3 - mape 50.15584172646621
alpha 0.4 - mape 51.456780665477275
alpha 0.5 - mape 52.83597082378954
alpha 0.6 - mape 54.33146015518122
alpha 0.7 - mape 55.984893194083455
alpha 0.8 - mape 57.830988295314526
alpha 0.9 - mape 60.03106668992228
best alpha 0.1 - mape is 47.18228759349405 - absolute error 543906.819112522 - forecast sum 1152777.5503354801
'2001100126405
46

alpha 0.1 - mape 71.58416670188514
alpha 0.2 - mape 96.35673256151969
alpha 0.3 - mape 101.47082482902299
alpha 0.4 - mape 102.93715799274659
alpha 0.5 - mape 104.6244593787526
alpha 0.6 - mape 107.1697012241897
alpha 0.7 - mape 110.6595577668956

alpha 0.7 - mape 86.60452646347197
alpha 0.8 - mape 83.85782141575466
alpha 0.9 - mape 80.8376996054752
best alpha 0.9 - mape is 80.8376996054752 - absolute error 2306886.9511416284 - forecast sum 2853726.618150056
'2001100166116
65

alpha 0.1 - mape 150.15431838829718
alpha 0.2 - mape 132.1012975237209
alpha 0.3 - mape 122.79863031133101
alpha 0.4 - mape 121.3676904116844
alpha 0.5 - mape 120.54857876818342
alpha 0.6 - mape 119.42469209514883
alpha 0.7 - mape 117.6916937613571
alpha 0.8 - mape 115.72506208427677
alpha 0.9 - mape 113.7051423060056
best alpha 0.9 - mape is 113.7051423060056 - absolute error 798000.5532703438 - forecast sum 701815.7113094749
'2001100166117
66

alpha 0.1 - mape 79.09316887846859
alpha 0.2 - mape 82.34535774451837
alpha 0.3 - mape 81.72867752960357
alpha 0.4 - mape 79.51686569292788
alpha 0.5 - mape 74.9005837504481
alpha 0.6 - mape 70.3532979277829
alpha 0.7 - mape 66.2296240559054
alpha 0.8 - mape 61.68625865170767
alpha 0.9 - mape 57.03889101106824
best

alpha 0.5 - mape 38.81669753102858
alpha 0.6 - mape 40.15744065969456
alpha 0.7 - mape 41.59894407787213
alpha 0.8 - mape 43.226862871559376
alpha 0.9 - mape 45.08209377515552
best alpha 0.1 - mape is 35.77627952755499 - absolute error 747.0458161681182 - forecast sum 2088.1036989683107
'8005700963801
89

alpha 0.1 - mape 60.32464725829482
alpha 0.2 - mape 63.223897298269584
alpha 0.3 - mape 60.67167763428174
alpha 0.4 - mape 57.91984909314738
alpha 0.5 - mape 57.35838279699526
alpha 0.6 - mape 57.52538196030128
alpha 0.7 - mape 58.752705773967165
alpha 0.8 - mape 60.59100566048276
alpha 0.9 - mape 62.479109217025986
best alpha 0.5 - mape is 57.35838279699526 - absolute error 479.1776385206844 - forecast sum 835.4099525724187
'8005700974001
90

alpha 0.1 - mape 100.25867683557641
alpha 0.2 - mape 94.0696210410114
alpha 0.3 - mape 92.6193233427724
alpha 0.4 - mape 94.96972753887272
alpha 0.5 - mape 101.13083114555195
alpha 0.6 - mape 110.72668349645585
alpha 0.7 - mape 122.9348072013788

alpha 0.5 - mape 91.56847159545939
alpha 0.6 - mape 94.80095136852252
alpha 0.7 - mape 97.11793481096407
alpha 0.8 - mape 98.34656259571122
alpha 0.9 - mape 98.36511387753471
best alpha 0.1 - mape is 78.39818373309708 - absolute error 264.8657331570639 - forecast sum 337.84677213797045
'80058014315601
109

alpha 0.1 - mape 109.9023732062727
alpha 0.2 - mape 104.51338226658254
alpha 0.3 - mape 103.62410471372314
alpha 0.4 - mape 103.02445232266741
alpha 0.5 - mape 102.23344878199524
alpha 0.6 - mape 101.99065011256624
alpha 0.7 - mape 104.74377353760174
alpha 0.8 - mape 108.76853754343739
alpha 0.9 - mape 112.90215983131233
best alpha 0.6 - mape is 101.99065011256624 - absolute error 184.28771352172774 - forecast sum 180.6907920660677
'80058014617001
110

alpha 0.1 - mape 84.60485318547944
alpha 0.2 - mape 82.82645948125848
alpha 0.3 - mape 87.66280477124242
alpha 0.4 - mape 94.0919724629242
alpha 0.5 - mape 100.54825355282524
alpha 0.6 - mape 106.98875293674675
alpha 0.7 - mape 113.577

alpha 0.1 - mape 113.98479641607923
alpha 0.2 - mape 116.74992649179698
alpha 0.3 - mape 119.16667330557422
alpha 0.4 - mape 116.04955717239571
alpha 0.5 - mape 111.94650812691877
alpha 0.6 - mape 111.90270409463679
alpha 0.7 - mape 111.5829048572314
alpha 0.8 - mape 111.25162540415441
alpha 0.9 - mape 111.01972883468329
best alpha 0.9 - mape is 111.01972883468329 - absolute error 51.15834030549086 - forecast sum 46.080404665435154
'80064018517601
135

alpha 0.1 - mape 101.85166902957177
alpha 0.2 - mape 124.87419070856383
alpha 0.3 - mape 148.7024238600625
alpha 0.4 - mape 167.53347883938747
alpha 0.5 - mape 180.4289774953033
alpha 0.6 - mape 188.9179371291022
alpha 0.7 - mape 194.50168805347406
alpha 0.8 - mape 198.08838989262517
alpha 0.9 - mape 200.12997007162548
best alpha 0.1 - mape is 101.85166902957177 - absolute error 56.91850405692361 - forecast sum 55.88372247527707
'80064018623301
136

alpha 0.1 - mape 77.97333271009562
alpha 0.2 - mape 78.51837871863886
alpha 0.3 - mape 79

alpha 0.1 - mape 71.06112466945414
alpha 0.2 - mape 72.46732216016201
alpha 0.3 - mape 75.85081632403472
alpha 0.4 - mape 78.2094782009924
alpha 0.5 - mape 80.2659136687627
alpha 0.6 - mape 82.53263284283014
alpha 0.7 - mape 86.93569859094869
alpha 0.8 - mape 93.0930751124756
alpha 0.9 - mape 100.07512794639828
best alpha 0.1 - mape is 71.06112466945414 - absolute error 1667.9948116640949 - forecast sum 2347.267678949483
'80075037020101
155

alpha 0.1 - mape 163.4118247041196
alpha 0.2 - mape 166.97646932489852
alpha 0.3 - mape 164.5696623810531
alpha 0.4 - mape 160.90517516187006
alpha 0.5 - mape 158.03981190089382
alpha 0.6 - mape 156.05888477283727
alpha 0.7 - mape 154.72395596464355
alpha 0.8 - mape 153.7970010364831
alpha 0.9 - mape 153.08732416796613
best alpha 0.9 - mape is 153.08732416796613 - absolute error 344.30373585895984 - forecast sum 224.90675680057785
'8007602451301
156

alpha 0.1 - mape 42.40053612023525
alpha 0.2 - mape 42.23202493621375
alpha 0.3 - mape 42.919176723

alpha 0.8 - mape 102.01737874182608
alpha 0.9 - mape 102.63905991007749
best alpha 0.1 - mape is 86.7629553073179 - absolute error 7686.741910038833 - forecast sum 8859.47451053515
'80082030020902
184

alpha 0.1 - mape 43.71167421915727
alpha 0.2 - mape 45.907502007576376
alpha 0.3 - mape 50.60479631647775
alpha 0.4 - mape 55.01361582420854
alpha 0.5 - mape 59.1851085429142
alpha 0.6 - mape 63.30601568008366
alpha 0.7 - mape 67.5298052226522
alpha 0.8 - mape 71.99768671028603
alpha 0.9 - mape 76.83845135573468
best alpha 0.1 - mape is 43.71167421915727 - absolute error 934.3423841805958 - forecast sum 2137.5122341370006
'80082039821501
185

alpha 0.1 - mape 94.82763134588079
alpha 0.2 - mape 94.50630770936297
alpha 0.3 - mape 93.69302256082295
alpha 0.4 - mape 93.6822561312866
alpha 0.5 - mape 93.3394792333577
alpha 0.6 - mape 92.25354544607785
alpha 0.7 - mape 92.37015052194238
alpha 0.8 - mape 91.89193365777616
alpha 0.9 - mape 90.52642545590925
best alpha 0.9 - mape is 90.5264254559

In [12]:
print (model)
df=pd.DataFrame(model)
print (df)
df.to_csv('SimpleExpo python ouput (DP26 all skus).csv', sep=',')

{'simpleExpo_mape': [37.41990296993081, 37.41990296993081, 19.79111593872228, 28.299548123795816, 101.55121669889841, 97.7447594408978, 60.53911990765906, 67.90602739266545, 133.22553453745002, 24.237096290122384, 33.41236434808807, 37.86766812918677, 30.715913079845592, 36.238737051379665, 30.481029104231038, 26.68275761485936, 30.55178808888957, 36.76576217252345, 46.24610810005806, 62.07326770565348, 131.06219571934653, 118.99371881366139, 71.84531384273197, 113.5762217391378, 102.89351718043514, 86.91179472653765, 74.45987245658361, 47.8577294651838, 63.13779369694308, 37.47454896390605, 57.72882837768955, 62.67853293497188, 26.915859468978798, 12.470535649047862, 26.742262996235095, 102.61618123154028, 16.60716660523586, 36.81659095333885, 31.285822778822727, 56.51216581146766, 57.32417209366852, 65.32293431042791, 65.13243571536327, 89.45338716652823, 92.50921698558886, 94.8286697918436, 47.18228759349405, 71.58416670188514, 29.741258181192613, 25.75714340812344, 22.0683016886984

In [ ]:
print (model2)
df1=pd.DataFrame(model2)
print (df1)